# Try to predict rho with the smallest turbulence dataset

In [1]:
import numpy as np
import os
import torch
from utils import *
from athena_read import *
from _arfima import arfima

## Read the data with athena_read

In [2]:
data_path='C:/Users/52673/Desktop/NYU MSDS/3-DS-1006 CAPSTONE/data_turb_dedt1_16'

In [3]:
# lets take a look at what a sample looks like
lst = sorted(os.listdir(data_path))[4:]
sample_path = data_path + '/' + lst[1]
sample = athdf(sample_path)
sample

{'Coordinates': b'cartesian',
 'DatasetNames': array([b'prim'], dtype='|S21'),
 'MaxLevel': 0,
 'MeshBlockSize': array([16, 16, 16]),
 'NumCycles': 2,
 'NumMeshBlocks': 1,
 'NumVariables': array([5]),
 'RootGridSize': array([16, 16, 16]),
 'RootGridX1': array([-0.5,  0.5,  1. ], dtype=float32),
 'RootGridX2': array([-0.5,  0.5,  1. ], dtype=float32),
 'RootGridX3': array([-0.5,  0.5,  1. ], dtype=float32),
 'Time': 0.008732914,
 'VariableNames': array([b'rho', b'press', b'vel1', b'vel2', b'vel3'], dtype='|S21'),
 'x1f': array([-0.5   , -0.4375, -0.375 , -0.3125, -0.25  , -0.1875, -0.125 ,
        -0.0625,  0.    ,  0.0625,  0.125 ,  0.1875,  0.25  ,  0.3125,
         0.375 ,  0.4375,  0.5   ], dtype=float32),
 'x1v': array([-0.46875, -0.40625, -0.34375, -0.28125, -0.21875, -0.15625,
        -0.09375, -0.03125,  0.03125,  0.09375,  0.15625,  0.21875,
         0.28125,  0.34375,  0.40625,  0.46875], dtype=float32),
 'x2f': array([-0.5   , -0.4375, -0.375 , -0.3125, -0.25  , -0.1875, -0.1

In [4]:
print('size of rho:', np.shape(sample['rho']))

size of rho: (16, 16, 16)


Q: Here the size of the matrix 'rho' for each timestep is (16,16,16). I interprete it like this: When I look into the setting that generate the dataset, I noticed that the mesh-block is set to be 16 for each axis x1, x2 and x3, so it is like cutting a large cube alone with its length, width, and height, each 16 times, and divide it into $16*16*16$ small cubes. So the matrix 'rho' represents the density of each small cube. 

Need to be confirmed.

## Extract rho and time from the original dataset

In [5]:
time = []
rho = []
for name in lst:
    path = data_path+'/'+name
    d = athdf(path)
    time.append(d['Time'])
    rho.append(d['rho'])

rho = np.array(rho)

In [6]:
# check whether the time is evenly distributed
time_gap = []
for i in range(1,len(time)):
    time_gap.append(time[i] - time[i-1])

print('gap between two time steps: ', time_gap)

gap between two time steps:  [0.008732914, 0.004386274, 0.004412315, 0.0044503715, 0.0044987686, 0.004519906, 0.004550522, 0.0045853965, 0.009305038, 0.004717216, 0.0047579333, 0.0048104264, 0.0048691407, 0.0049307495, 0.0049957708, 0.005053267, 0.0051188394, 0.0051931813, 0.0052760392, 0.0053668246, 0.005464494, 0.005568497, 0.0055926517, 0.005605586, 0.005638182, 0.0056675375, 0.0056544095, 0.005655691, 0.0056724995, 0.0057038367, 0.005744025, 0.005776748, 0.005776122, 0.00578624, 0.0058072805, 0.005786702, 0.0057302415, 0.005693823, 0.0056090504, 0.005554706, 0.005525619, 0.0055146664, 0.0055193305, 0.00553903, 0.0055715293, 0.0056138337, 0.0056639016, 0.0057087243, 0.005660951, 0.0056361556, 0.005596459, 0.005533755, 0.0055256784, 0.0055434406, 0.0055660605, 0.005599469, 0.005648643, 0.005661726, 0.00566715, 0.005690187, 0.0057277083, 0.0057603717, 0.005774826, 0.0058030784, 0.0058419406, 0.005887121, 0.005936116, 0.0059885383, 0.0060466826, 0.006113082, 0.0061888993, 0.006265551, 

Q: Here we notice that the time is not evenly distributed, so we might want to encode time into the transformer as well? But how?

## Preprocess on rho

In [7]:
np.shape(rho)

(161, 16, 16, 16)

In [8]:
rho_reshaped = np.reshape(rho, (-1, 16))
np.shape(rho_reshaped)

(41216, 16)

In [9]:
# check that the reshape does not lose the positional information
x1 = 5
x2 = 1
x3 = 3

print(rho[x1][x2][x3])
print(rho_reshaped[x1*256+x2*16+x3])

[0.94767326 0.9405406  0.954838   0.97035104 0.98924935 1.0113069
 1.0361519  1.0550262  1.078851   1.1301337  1.2055086  1.2523041
 1.2115914  1.1034071  1.0169045  0.98763895]
[0.94767326 0.9405406  0.954838   0.97035104 0.98924935 1.0113069
 1.0361519  1.0550262  1.078851   1.1301337  1.2055086  1.2523041
 1.2115914  1.1034071  1.0169045  0.98763895]


In [10]:
[0+256*i for i in range(10)]

[0, 256, 512, 768, 1024, 1280, 1536, 1792, 2048, 2304]

In [11]:
def to_windowed(data,window_size,pred_size):
    out = []
    for i in range(len(data)-256*window_size):
        feature  = np.array(data[[i+256*k for k in range(window_size)]])
        target = np.array(data[[i+256*(k+pred_size) for k in range(window_size)]])        
        out.append((feature,target))
    return np.array(out)

In [12]:
np.shape(to_windowed(rho_reshaped,10,1))

(38656, 2, 10, 16)

### Use the edited ultils function

In [84]:
train,val,test,train_data,val_data,test_data = train_test_val_split(rho_reshaped ,train_proportion = 0.5, val_proportion = 0.25, test_proportion = 0.25\
              , window_size = 10, pred_size = 1)

In [85]:
dataset_train = CustomDataset(train)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=16, 
                                            drop_last=False, 
                                            num_workers=1, pin_memory=True)

In [86]:
for i,(data, targets) in enumerate(train_loader):  
            data, targets = data, targets

In [87]:
targets.shape

torch.Size([16, 10, 16])

this match with the required size to feed into the transformer (batch_size, sequence_length, feature_size)